<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/Retrieval_augmented_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
generative_model="meta-llama/Llama-3.2-1B-Instruct"
sentence_model = "dmis-lab/biobert-base-cased-v1.1"

In [22]:
from google.colab import userdata

In [23]:
!pip install langchain_huggingface
!pip install -qU langchain-text-splitters
!pip install -qU "langchain-chroma>=0.1.2"
!pip install -U langchain-community

In [24]:
!wget https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json

--2024-12-11 12:28:00--  https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2584787 (2.5M) [text/plain]
Saving to: ‘ori_pqal.json.1’

ori_pqal.json.1     100%[===================>]   2.46M  --.-KB/s    in 0.05s   

2024-12-11 12:28:01 (45.5 MB/s) - ‘ori_pqal.json.1’ saved [2584787/2584787]



We collect two datasets:

    ‘questions’: the questions with corresponding gold long answer, gold document ID, and year.
    ‘documents’: the abstracts (contexts+long_answer concatenated), and year.


In [25]:
import pandas as pd
tmp_data = pd.read_json("/content/ori_pqal.json").T
# some labels have been defined as "maybe", only keep the yes/no answers
tmp_data = tmp_data[tmp_data.final_decision.isin(["yes", "no"])]

documents = pd.DataFrame({"abstract": tmp_data.apply(lambda row: (" ").join(row.CONTEXTS+[row.LONG_ANSWER]), axis=1),
             "year": tmp_data.YEAR})
questions = pd.DataFrame({"question": tmp_data.QUESTION,
             "year": tmp_data.YEAR,
             "gold_label": tmp_data.final_decision,
             "gold_context": tmp_data.LONG_ANSWER,
             "gold_document_id": documents.index})

For an example of a query:

In [26]:
questions.iloc[0].question

'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?'

For an example of a document to leverage for the queries:

In [27]:
documents.iloc[0].abstract

'Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos a

In [28]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=sentence_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [30]:
texts = text_splitter.create_documents([documents.iloc[0].abstract])
print(texts[0])
print(texts[1])
text_splitter.split_text(documents.iloc[0].abstract)[:2]

page_content='Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant'
page_content='The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The'


['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant',
 'The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The']

If you want to get best in-class automated tracing of your model calls you can also set your LangSmith API key by uncommenting below:

In [31]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

In [67]:

for i, doc in enumerate(documents.iloc[:10].values):
  print(ids[i], doc[0])

21645374 Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker Red 

In [76]:
from uuid import uuid4

from langchain_core.documents import Document

split_idx = 10
ids = documents.iloc[:split_idx].index

docs=[]
for i, doc in enumerate(documents.iloc[:split_idx].values):
  temp_doc = Document(
      page_content=doc[0],
      metadata={"source": "tweet"},
      id=ids[i],
  )
  print(temp_doc)
  print(temp_doc.id)
  docs.append(temp_doc)

page_content='Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker

In [84]:
from langchain.vectorstores import Chroma

try:
  vector_store.delete_collection()
except:
  pass

vector_store = Chroma.from_documents(docs, hf)
#retriever = vector_store.as_retriever(...)

In [85]:
print(vector_store._collection.count())

10


In [86]:
results = vector_store.similarity_search_with_score(
    "What is programmed cell death?", k=3
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=46.547935] Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTra

In [36]:
from huggingface_hub import login
login(userdata.get('huggingface_token'))

In [37]:
! huggingface-cli download $generative_model --local-dir ./$generative_model

Fetching 13 files:   0% 0/13 [00:00<?, ?it/s]
README.md: 100% 41.7k/41.7k [00:00<00:00, 13.8MB/s]
Download complete. Moving file to meta-llama/Llama-3.2-1B-Instruct/README.md
LICENSE.txt: 100% 7.71k/7.71k [00:00<00:00, 37.2MB/s]
Download complete. Moving file to meta-llama/Llama-3.2-1B-Instruct/LICENSE.txt
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_http.py", line 406, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/9213176726f574b556790deb65791e0c5aa438b6/.gitattributes

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    s

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("./"+generative_model)
model_hf = AutoModelForCausalLM.from_pretrained("./"+generative_model)

OSError: Can't load tokenizer for './meta-llama/Llama-3.2-1B-Instruct'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './meta-llama/Llama-3.2-1B-Instruct' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.

In [ ]:
retriever = vector_store.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_hf,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device="cuda"
    )

# Wrap the pipeline in a LangChain HuggingFacePipeline object
llm = HuggingFacePipeline(pipeline=pipe)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = retrieval_chain.invoke("Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?")

In [ ]:

# Assuming the output is a dictionary or a similar structure:
if isinstance(response, dict):
    for key, value in response.items():
        print(f"{key}: {value}")
elif isinstance(response, list):
    for idx, item in enumerate(response):
        print(f"Item {idx + 1}: {item}")
else:
    print(response)


In [ ]:
from tqdm import tqdm
template = """Answer the question, only with yes or no, based only on the following context:
{context}
Do not write anything else. Do not explain your answer.
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_hf,
    tokenizer=tokenizer,
    max_new_tokens=10,
    device="cuda"
    )

# Wrap the pipeline in a LangChain HuggingFacePipeline object
llm = HuggingFacePipeline(pipeline=pipe)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Initialize a counter for correct answers
correct_count = 0
q_a = zip(questions.question, questions.gold_label)
print(len(questions.question))
for question, gold_label in q_a:
    answer = retrieval_chain.invoke(question)
    print(type(answer))
    y_n = answer.splitlines()[-1].split()[0]
    # Check if the answer is "yes" or "no" and matches the gold label
    if gold_label.lower() in answer[-1].lower():
        print("Correct")
        correct_count += 1
    else:
        print("Incorrect")

# Print the total number of correct answers
print(f"Total correct answers: {correct_count}")


